# Image Classification using the pre-trained ImageNet-1k dataset

### To use the pretrained models using ImageNet 1000 for image classification
### Trained Dataset: ImageNet 1000

### - AI Tool: pytorch, torchvision
### - Use pretrained models 
### - models: ResNet50, VGG16, VGG19
### - Reference 
  - [torchvision.models](https://docs.pytorch.org/vision/main/models.html):
    [ResNet50](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.resnet50.html),
    [VGG19](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.vgg19.html#torchvision.models.vgg19), 
    [VGG16](https://docs.pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html#torchvision.models.vgg16)
  - imagenet 1000 class list: 
     [keras-imagenet_class_index.json](https://github.com/raghakot/keras-vis/blob/master/resources/imagenet_class_index.json),
     [Class ID-Class Name Table](https://deeplearning.cms.waikato.ac.nz/user-guide/class-maps/IMAGENET/),                                  [clsidx_to_lables.txt](https://gist.github.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57)
 
  - imagenet-1k Dataset card: https://huggingface.co/datasets/imagenet-1k
  - image-net.org: https://www.image-net.org/update-mar-11-2021.php
  - [Using keras: image classifications](https://keras.io/api/applications/)
 
## **실습** 
  - image classification을 알아보기
  - input, output, preprocess, model, inference(predict) 이해하기
  - web에 있는 image로 test해 보세요(검색, browsing사용).
    * 단, 접속되는 site가 제한될 수 있으니, 접속이 안되면 다른 site를 시도해 보세요.
  - model architecture 보기 

In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms.functional as F
from torchvision.io import decode_image

from torchinfo import summary

import numpy as np
import matplotlib.pyplot as plt

import os
import sys

## Test Images

In [ ]:
# test images 포함한 폴더
img_path = './images/'

In [ ]:
if sys.platform == 'win32':
    # path change to windows
    img_path = '.\\images\\'
    !dir {img_path} /B 
else:
    !ls $img_path

### load an image from url

In [ ]:
# load image from url
import urllib
from io import BytesIO
from PIL import Image
import requests

def load_image_url(url):
    return Image.open(requests.get(url, stream=True).raw) # PIL format
    
def load_image_url2(URL):
    with urllib.request.urlopen(URL) as url:
        img_file = BytesIO(url.read())
        img = Image.open(img_file)

    return img # PIL format

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Load a model for image classification

In [ ]:
weights = models.ResNet50_Weights.DEFAULT
resnet50 = models.resnet50(weights=weights)
#resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

### Loading an image

In [ ]:
# 1. load from an image in the local directory
img_file = img_path + 'elephant.jpg'
img = decode_image(img_file) #,  # Loads an image into PIL format
img = F.to_pil_image(img.detach())
plt.imshow(img)
plt.axis('off')

In [ ]:
img

### image classification given an image
- inference or predict

In [ ]:
resnet50.eval()

preprocess = weights.transforms()

x = preprocess(img).unsqueeze(0)
preds = resnet50(x).squeeze(0).softmax(0)
class_id = preds.argmax().item()
score = preds[class_id].item()
category_name = weights.meta['categories'][class_id]
print(f'Predicted: {category_name}: {score:.1f}')

topk_probs, topk_class_ids = torch.topk(preds, k=3)
print(f'Predicted:', end='') 
for p, class_id in zip(topk_probs, topk_class_ids):
    print(f' {weights.meta['categories'][class_id]}({p:.1f}),', end='')
print('') 

In [ ]:
img.size, x.shape 

In [ ]:
# 2. url에서 직접 download하여 test
img_url1 = 'https://d1bg8rd1h4dvdb.cloudfront.net/img/storypick/monamipet/2019/01/1811_pet_dog_pomeranian_m_01.jpg'
img_url2 = 'https://github.com/pytorch/hub/raw/master/images/dog.jpg'
img1 = load_image_url(img_url1)
img2 = load_image_url(img_url2)
plt.subplot(121)
plt.imshow(img1)
plt.axis('off')
plt.subplot(122)
plt.imshow(img2)
plt.axis('off')

In [ ]:
def predict(model, weights, img):
    # weights: are sused for preprocess(image), meta['cateogries'][class_id] (category name) 
    model.eval()

    preprocess = weights.transforms() # preprocess_input

    x = preprocess(img).unsqueeze(0)
    preds = model(x).squeeze(0).softmax(0)
    class_id = preds.argmax().item()
    score = preds[class_id].item()
    category_name = weights.meta['categories'][class_id]

    topk_probs, topk_class_ids = torch.topk(preds, k=3)
    print (topk_probs)
    print(f'Predicted:', end='') 
    for p, class_id in zip(topk_probs, topk_class_ids):
        print(f' {weights.meta['categories'][class_id]}({p:.3f}),', end='')

In [ ]:
predict(resnet50, weights, img1) 

In [ ]:
predict(resnet50, weights, img2) 

## Model 보기

In [ ]:
resnet50

In [ ]:
summary(resnet50, [1, 3, 224, 224])

In [ ]:
dir(weights)

## Use VGG16, VGG19


In [ ]:
vgg19_weights = models.VGG19_Weights.IMAGENET1K_V1
vgg16_weights = models.VGG16_Weights.IMAGENET1K_V1
vgg19_model = models.vgg19(weights=vgg19_weights)
vgg16_model = models.vgg16(weights=vgg16_weights)

In [ ]:
predict(vgg16_model, vgg16_weights, img1)

In [ ]:
predict(vgg19_model, vgg19_weights, img1)

In [ ]:
predict(vgg19_model, vgg19_weights, img2)

In [ ]:
predict(vgg16_model, vgg19_weights, img2)

In [ ]:
vgg19_model

In [ ]:
summary(vgg19_model, input_size=[1, 3, 224, 224])

In [ ]:
dir(models.VGG19_Weights.IMAGENET1K_V1)

# [참고]
## MobileNet v2, v3

### Use VGG16, VGG19
- You may use to extract features with VGG16
